In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from datetime import date
import holidays



print("Setup Complete")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction
before create model I did EDA with other notebook here (https://www.kaggle.com/imnaho/tps-jan2022-simple-eda)

from above notebook I found

**What will I predict for this competition?**
* how many number would be sold at each country ,store ,product in a day in 2019

**What is nice feature to predict the number of sold ?**
* It is increasing year by year and new year holiday season is most high number of sold
* each country has almost same feature with Time series graph
* December sold most,April is second, January is third
* when I look at the number of sold only Kaggle Hat, it sold most in April ,not December
* Saturday and Sunday sold more than other day
* almost same but day of 31 is sold most

In [ ]:
train_original = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test_original  = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
train=train_original.copy()
test=test_original.copy()

# check data

at my EDA notebook it is known 
* they have no missing value
* training data is only object data when i remove target and id columns (so i have to encode these data to create model and predict)

In [ ]:
print('---------train data types is here-----------')
print(train.dtypes)
print('---------train data shape is here-----------')
print(train.shape)
print('---------test data types is here-----------')
print(test.dtypes)
print('---------test data shape is here-----------')
print(test.shape)
print('---------train data nique is here-----------')
print(train.nunique())
print('---------test data nique is here-----------')
print(test.nunique())

# encode data

split date to year,month,day

factorize cateorical data country,store,product

In [ ]:
for df in [train, test]:
    df[['year', 'month','day']] = (df["date"].str.split("-", expand=True))
    df[['year', 'month','day']] = df[['year', 'month','day']].astype(int)
    df['date'] = pd.to_datetime(df.date)
    df['day_of_week'] = df["date"].dt.weekday
    df['country'], country = df['country'].factorize()
    df['store'], store = df['store'].factorize()
    df['product'], product = df['product'].factorize()

In [ ]:
print('---------train data types is here-----------')
print(train.dtypes)
print('---------test data types is here-----------')
print(test.dtypes)

In [ ]:
train.head()

# add some features

December sold most,April is second, January is third so I'll add this feature

In [ ]:
train['more_sold_month'] = 0
train.loc[train['month']==12,'more_sold_month'] = 3
train.loc[train['month']==4,'more_sold_month']  = 2
train.loc[train['month']==1,'more_sold_month']  = 1
train.head()

when I look at the number of sold only Kaggle Hat, it sold most in April ,not December

so I'll add "KaggleHat" columns 

In [ ]:
train['KaggleHat'] = 0
train.loc[train['product']==1,'KaggleHat'] = 1
train.head()

Saturday and Sunday sold more than other day

In [ ]:
train['Sat_or_Sun'] = 0
train.loc[train['day_of_week']==5,'Sat_or_Sun'] = 1
train.loc[train['day_of_week']==6,'Sat_or_Sun'] = 1
train.head()

It is increasing year by year and new year holiday season is most high number of sold

In [ ]:
train.loc[train['year']==2015,'year'] = 1
train.loc[train['year']==2016,'year'] = 2
train.loc[train['year']==2017,'year'] = 3
train.loc[train['year']==2018,'year'] = 4
train.head()

almost same but day of 31 is sold most and 30,29,28 is little bit more sold than other days

In [ ]:
train['more_sold_day'] = 0
train.loc[train['day']==31,'more_sold_day'] = 2
train.loc[train['day']==30,'more_sold_day'] = 1
train.loc[train['day']==29,'more_sold_day'] = 1
train.loc[train['day']==28,'more_sold_day'] = 1
train.head()

# separate data to train_data and valid_data

In [ ]:
from sklearn.model_selection import train_test_split
X=train.drop(['row_id','date','num_sold'],axis = 1)
y = train.num_sold

#Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.15,random_state=0)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

# create simple model with XGBRegressor

In [ ]:
def df_XGBRegressor(X_train, X_valid, y_train, y_valid):
    my_model = XGBRegressor(n_estimators=1000, learning_rate=0.02,random_state=0)
    my_model.fit(X_train, y_train,early_stopping_rounds=5, eval_set=[(X_valid, y_valid)],verbose=False,
                 eval_metric='mae')
    predict = my_model.predict(X_valid)
    return predict,my_model

In [ ]:
predict,simple_model =df_XGBRegressor(X_train, X_valid, y_train, y_valid)

# check score

In [ ]:
def get_smap(predict,y_valid):
    smape = 100/len(y_valid) * np.sum(2 * np.abs(predict - y_valid) / (np.abs(predict) + np.abs(y_valid)))
    return smape

In [ ]:
print('smape score is : ',get_smap(predict,y_valid))

# make prediction for submit

prepare test data

In [ ]:
test['more_sold_month'] = 0
test.loc[test['month']==12,'more_sold_month'] = 3
test.loc[test['month']==4,'more_sold_month']  = 2
test.loc[test['month']==1,'more_sold_month']  = 1


test['KaggleHat'] = 0
test.loc[test['product']==1,'KaggleHat'] = 1


test['Sat_or_Sun'] = 0
test.loc[test['day_of_week']==5,'Sat_or_Sun'] = 1
test.loc[test['day_of_week']==6,'Sat_or_Sun'] = 1


test.loc[test['year']==2019,'year'] = 5

test['more_sold_day'] = 0
test.loc[test['day']==31,'more_sold_day'] = 2
test.loc[test['day']==30,'more_sold_day'] = 1
test.loc[test['day']==29,'more_sold_day'] = 1
test.loc[test['day']==28,'more_sold_day'] = 1


In [ ]:
test_data=test.drop(['row_id','date'],axis = 1)
int_predict = np.round(simple_model.predict(test_data))
int_predict = int_predict.astype(int)
output = pd.DataFrame({'row_id': test.row_id, 'num_sold': int_predict})
output.to_csv('submission.csv', index=False)

In [ ]:
output.head()